In [1]:
# step 0 installs
%pip install transformers torch pandas numpy matplotlib networkx seaborn


  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)
  Using cached pandas-2.0.3-cp38-cp38-win_amd64.whl.metadata (18 kB)
  Using cached matplotlib-3.7.5-cp38-cp38-win_amd64.whl.metadata (5.8 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached PyYAML-6.0.3-cp38-cp38-win_amd64.whl.metadata (2.2 kB)
  Using cached regex-2024.11.6-cp38-cp38-win_amd64.whl.metadata (41 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached tokenizers-0.20.3-cp38-none-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached contourpy-1.1.1-cp38-cp38-win_amd64.whl.metadata (5.9 kB)
  Using cached cycler-0.12.

In [4]:
%pip install scikit-learn


  Using cached scikit_learn-1.3.2-cp38-cp38-win_amd64.whl.metadata (11 kB)
  Using cached scipy-1.10.1-cp38-cp38-win_amd64.whl.metadata (58 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.3.2-cp38-cp38-win_amd64.whl (9.3 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.10.1-cp38-cp38-win_amd64.whl (42.2 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


In [5]:
# step 0 imports
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
import random
import seaborn as sns

In [ ]:
# step 1: Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("InstaDeepAI/nucleotide-transformer-2.5b-multi-species")
model = AutoModelForMaskedLM.from_pretrained("InstaDeepAI/nucleotide-transformer-2.5b-multi-species")

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.08s/it]


In [ ]:
# step 2 function to parse FASTA files
def parse_fasta(file_path):
    with open(file_path) as f:
        seq = []

        for line in f:
            line = line.strip()
            if line.startswith('>'):
                continue
            else:
                seq.append(line)
    return seq



In [ ]:
# step 3 parse sequences from data file
strain_1 = parse_fasta('data\GCA_006094915.1\GCA_006094915.1_ASM609491v1_genomic.fna')[0]
strain_2 = parse_fasta('data\GCA_026167765.1\GCA_026167765.1_ASM2616776v1_genomic.fna')[0]
strain_3 = parse_fasta('data\GCA_900607265.1\GCA_900607265.1_BPH2003_genomic.fna')[0]
strain_4 = parse_fasta('data\GCA_900620245.1\GCA_900620245.1_BPH2947_genomic.fna')[0]

In [23]:
# step 4: Define masked embedding function
def get_masked_embedding(sequence):
    tokens = tokenizer(sequence, return_tensors="pt", truncation=True, padding="max_length", max_length=tokenizer.model_max_length)
    input_ids = tokens["input_ids"]
    attention_mask = input_ids != tokenizer.pad_token_id

    with torch.no_grad():
        outputs = model(
            input_ids,
            attention_mask=attention_mask,
            encoder_attention_mask=attention_mask,
            output_hidden_states=True
        )
    embeddings = outputs.hidden_states[-1]
    attention_mask = attention_mask.unsqueeze(-1)
    masked_embeddings = embeddings * attention_mask
    mean_embedding = masked_embeddings.sum(dim=1) / attention_mask.sum(dim=1)
    return mean_embedding.squeeze().numpy()

In [ ]:
embedding_for_strain_1 = get_masked_embedding(strain_1)
embedding_for_strain_2 = get_masked_embedding(strain_2)
embedding_for_strain_3 = get_masked_embedding(strain_3)
embedding_for_strain_4 = get_masked_embedding(strain_4) 